# Dataset transformation for LWU San Jacinto Readings Dataset

In this notebook, the consolidated dataset is transformed into wide format, containing the cleaned consumption data.

In [1]:
from pathlib import Path
import os
import pandas as pd

base_dir = Path("../../dataset/clean/pre_production")
years = os.listdir(base_dir)

columns = ['Control Number', 'Cleaned Consumption']
all_data = []

for year in years:
    files = os.listdir(base_dir / year)
    for file in files:
        month = file.split("_")[0]
        df = pd.read_csv(base_dir / year / file, usecols=columns)
        df['Month_Year'] = f"{year}_{month}"
        all_data.append(df)

# Combine into a single DataFrame
df_all = pd.concat(all_data, ignore_index=True)

# Pivot to wide format
wide_df = df_all.pivot_table(
    index=['Control Number'],
    columns='Month_Year',
    values='Cleaned Consumption',
).reset_index()

wide_df = wide_df.sort_index(axis=1)

# Optional: Sort columns so months are in order
print(wide_df.head())


Month_Year  2019_DEC  2020_APR  2020_AUG  2020_DEC  2020_FEB  2020_JAN  \
0                NaN       NaN       NaN       NaN       NaN       NaN   
1                NaN       NaN       NaN       NaN       NaN       NaN   
2                NaN       NaN       NaN       NaN       NaN       NaN   
3                0.0       0.0       0.0       0.0       0.0       0.0   
4                0.0       0.0       0.0       0.0       0.0       0.0   

Month_Year  2020_JUL  2020_JUN  2020_MAR  2020_MAY  ...  2024_NOV  2024_OCT  \
0                NaN       NaN       NaN       NaN  ...       NaN       NaN   
1                NaN       NaN       NaN       NaN  ...       0.0       0.0   
2                NaN       NaN       NaN       NaN  ...       NaN       NaN   
3                0.0       0.0       0.0       0.0  ...       0.0       0.0   
4                0.0       0.0       0.0       0.0  ...      20.0       0.0   

Month_Year  2024_SEP  2025_APR  2025_FEB  2025_JAN  2025_MAR  2025_MAY  \
0     

In [4]:
# ================================================
# Consolidate Wide Data with Connection Status and Last Reading
# ================================================

import calendar
import os
from utilities import read_csv, save_csv

# ------------------------------------------------
# 1. Read source datasets
# ------------------------------------------------
right_df = read_csv("../../dataset/clean/pre_production/2025/SEP_2025_pre_production.csv")  
name_df = read_csv("../../dataset/clean/semi_clean/cleaned_master.csv") 

# ------------------------------------------------
# 2. Merge connection status from pre-production
# ------------------------------------------------
final_df = wide_df.merge(
    right_df[['Control Number', 'Connection Status']],
    on='Control Number',
    how='left'
)

# Normalize connection status values
final_df['Connection Status'] = (
    final_df['Connection Status']
    .replace({"Invalid Reading": "Active"})  # Convert "Invalid Reading" to "Active"
    .fillna("Unknown")                       # Fill missing with "Unknown"
)

# ------------------------------------------------
# 3. Identify month columns (exclude non-monthly fields)
# ------------------------------------------------
fixed_cols_pre_sort = [col for col in final_df.columns if not col[0].isdigit()]  # Columns that are not YYYY_MON format
month_cols = [col for col in final_df.columns if col not in fixed_cols_pre_sort]

# ------------------------------------------------
# 4. Sort month columns chronologically
# ------------------------------------------------
month_map = {m.upper(): i for i, m in enumerate(calendar.month_abbr) if m}  # {"JAN":1, "FEB":2, ...}

month_cols_sorted = sorted(
    month_cols,
    key=lambda x: (int(x.split('_')[0]), month_map[x.split('_')[1].upper()])
)

# ------------------------------------------------
# 5. Compute the "Last Reading" (latest non-null value per row)
# ------------------------------------------------
final_df["Last Reading"] = final_df[month_cols_sorted].apply(
    lambda row: row.dropna().iloc[-1] if row.dropna().size > 0 else None,
    axis=1
)

# ------------------------------------------------
# 6. Merge account names from master list
# ------------------------------------------------
final_df = final_df.merge(
    name_df[['Control Number', 'Account Name', 'Connection Type', 'District', 'Street']],
    on='Control Number',
    how='left'
)

# ------------------------------------------------
# 7. Final column ordering: fixed fields first, then months
# ------------------------------------------------
fixed_cols_final = ['Control Number', 'Account Name', 'District', 'Street', 'Connection Type', 'Connection Status', 'Last Reading']
final_df = final_df[fixed_cols_final + month_cols_sorted]

# ------------------------------------------------
# 8. Save the consolidated output
# ------------------------------------------------
save_path = "../../dataset/clean/production/tabular/consumption_consolidated_2.csv"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
save_csv(final_df, save_path)

# ------------------------------------------------
# 9. Preview output for sanity check
# ------------------------------------------------
display(final_df.head())
display(final_df.info())


,Control Number,Account Name,District,Street,Connection Type,Connection Status,Last Reading,2019_DEC,2020_JAN,2020_FEB,...,2024_SEP,2024_OCT,2024_NOV,2024_DEC,2025_JAN,2025_FEB,2025_MAR,2025_APR,2025_MAY,2025_SEP
0,0.0,Kho Shipping Lines,NaN,Bailon Street,NaN,Unknown,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,121.0,"Almodal, Maricel",District 2,Esparrago Street,Residential,Disconnected,0.0,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50964.0,"Esparraguerra, Irish",District 2,Fabmar Street,Residential,Unknown,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,500001.0,"Encinares, Roberto",District 4,Esparrago Street,Residential,Disconnected,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,500002.0,"Pareja, Myrna",District 4,Esparrago Street,Residential,Active,153.0,0.0,0.0,0.0,...,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,153.0,153.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2180 entries, 0 to 2179
Data columns (total 62 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Control Number     2180 non-null   float64
 1   Account Name       2078 non-null   object 
 2   District           1733 non-null   object 
 3   Street             2078 non-null   object 
 4   Connection Type    1945 non-null   object 
 5   Connection Status  2180 non-null   object 
 6   Last Reading       2180 non-null   float64
 7   2019_DEC           1558 non-null   float64
 8   2020_JAN           1563 non-null   float64
 9   2020_FEB           1606 non-null   float64
 10  2020_MAR           1619 non-null   float64
 11  2020_APR           1619 non-null   float64
 12  2020_MAY           1621 non-null   float64
 13  2020_JUN           1619 non-null   float64
 14  2020_JUL           1621 non-null   float64
 15  2020_AUG           1627 non-null   float64
 16  2020_SEP           1694 

None